## Initialization

In [ ]:
import pandas as pd

In [ ]:
years = list(range(1999, 2024))

In [ ]:
csv_files = [f"./Data/play_by_play_{year}.csv" for year in years]

In [ ]:
# load each csv file as a dataframe and collect them in a list
dataframes = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dataframes.append(df)

## Removal of Useless Observations
Thilo & Tok

In [ ]:
list(dataframes[0]["play_type"].unique())

In [ ]:
# remove useless plays from dataframes to reduce size for combining dataframes
for dataframe in dataframes:
    dataframe.drop(
        dataframe[~dataframe["play_type"].isin(["pass", "run"])].index,
        axis=0,
        inplace=True,
    )

In [ ]:
# combine all dataframes into a single one
combined_df = pd.concat(dataframes, axis=0)

# reset the index of the combined dataframe
combined_df.reset_index(drop=True, inplace=True)

In [ ]:
# drop safeties
combined_df.drop(combined_df[combined_df["safety"] == 1].index, axis=0, inplace=True)

In [ ]:
# drop plays with penalties
combined_df.drop(combined_df[combined_df["penalty"] == 1].index, axis=0, inplace=True)

In [ ]:
# drop plays with laterals
combined_df.drop(
    combined_df[combined_df["lateral_reception"] == 1].index, axis=0, inplace=True
)

In [ ]:
# drop plays with laterals
combined_df.drop(
    combined_df[combined_df["lateral_rush"] == 1].index, axis=0, inplace=True
)

In [ ]:
# drop plays with replays or challenges
combined_df.drop(
    combined_df[combined_df["replay_or_challenge"] == 1].index, axis=0, inplace=True
)

In [ ]:
# drop two point conversion plays
combined_df.drop(
    combined_df[~combined_df["two_point_conv_result"].isna()].index, axis=0, inplace=True
)

## Handling of Missing Values
Tok

In [ ]:
display(combined_df[(combined_df['roof'].isna())].groupby('game_id').count())

In [ ]:
"""
https://www.nfl.com/games/jaguars-at-texans-2021-reg-1 (closed)
https://www.nfl.com/games/eagles-at-falcons-2021-reg-1 (open)
https://www.nfl.com/games/seahawks-at-colts-2021-reg-1 (closed)
https://www.nfl.com/games/rams-at-colts-2021-reg-2 (closed)
https://www.nfl.com/games/panthers-at-texans-2021-reg-3 (closed)
https://www.nfl.com/games/football-team-at-falcons-2021-reg-4 (open)
https://www.nfl.com/games/patriots-at-texans-2021-reg-5  (closed)
https://www.nfl.com/games/texans-at-colts-2021-reg-6 (open)
https://www.nfl.com/games/panthers-at-falcons-2021-reg-8 (open)
https://www.nfl.com/games/rams-at-texans-2021-reg-8 (closed)
https://www.nfl.com/games/titans-at-colts-2021-reg-8 (open)
https://www.nfl.com/games/jets-at-colts-2021-reg-9 (closed)
https://www.nfl.com/games/jaguars-at-colts-2021-reg-10 (closed)
https://www.nfl.com/games/patriots-at-falcons-2021-reg-11 (closed)
https://www.nfl.com/games/jets-at-texans-2021-reg-12 (closed)
https://www.nfl.com/games/buccaneers-at-colts-2021-reg-12 (closed)
https://www.nfl.com/games/colts-at-texans-2021-reg-13 (closed)
https://www.nfl.com/games/buccaneers-at-falcons-2021-reg-13 (closed)
https://www.nfl.com/games/seahawks-at-texans-2021-reg-14 (open)
https://www.nfl.com/games/patriots-at-colts-2021-reg-15 (closed)
https://www.nfl.com/games/lions-at-falcons-2021-reg-16 (open)
https://www.nfl.com/games/chargers-at-texans-2021-reg-16 (closed)
https://www.nfl.com/games/raiders-at-colts-2021-reg-17 (closed)
https://www.nfl.com/games/saints-at-falcons-2021-reg-18 (closed)
https://www.nfl.com/games/titans-at-texans-2021-reg-18 (closed)
"""


closed_roof = [
    "2021_01_JAX_HOU",
    "2021_01_SEA_IND",
    "2021_02_LA_IND",
    "2021_03_CAR_HOU",
    "2021_05_NE_HOU",
    "2021_08_LA_HOU",
    "2021_09_NYJ_IND",
    "2021_10_JAX_IND",
    "2021_11_NE_ATL",
    "2021_12_NYJ_HOU",
    "2021_12_TB_IND",
    "2021_13_IND_HOU",
    "2021_13_TB_ATL",
    "2021_15_NE_IND",
    "2021_16_LAC_HOU",
    "2021_17_LV_IND",
    "2021_18_NO_ATL",
    "2021_18_TEN_HOU"
]

open_roof = [
    "2021_01_PHI_ATL",
    "2021_04_WAS_ATL",
    "2021_06_HOU_IND",
    "2021_08_CAR_ATL",
    "2021_08_TEN_IND",
    "2021_14_SEA_HOU",
    "2021_16_DET_ATL"
]

def update_roof(row):
    if any(game_id in row['game_id'] for game_id in open_roof):
        return "open"
    if any(game_id in row['game_id'] for game_id in closed_roof):
        return "closed"
    return row['roof']

# Apply the function to update 'roof'
combined_df.loc[combined_df['roof'].isna(), 'roof'] = combined_df[combined_df['roof'].isna()].apply(update_roof, axis=1)



In [ ]:
display(combined_df[(combined_df['roof'].isna())].groupby('game_id').count())

## Dropping of Useless Features
Thilo & Tok

In [ ]:
drop_columns1 = [
    "play_id",
    "game_id",
    "old_game_id",
    "home_team",
    "away_team",
    "season_type",
    "week",
    # "defteam",
    # "posteam",
    # "posteam_type",
    "side_of_field",
    # "game_date",
    "quarter_seconds_remaining",
    "quarter_end",
    "drive",
    "sp",  # eventuell relevant laut TOK
    "qtr",
    "time",
    "yrdln",
    "ydsnet",
    "qb_kneel",
    "qb_spike",
    "pass_length",
    "pass_location",
    "run_location",
    "run_gap",
    "field_goal_result",
    "kick_distance",
    "extra_point_result",
    "two_point_conv_result",
    "home_timeouts_remaining",
    "away_timeouts_remaining",
    "timeout",
    "timeout_team",
    "td_team",
    "td_player_name",
    "td_player_id",
    "posteam_timeouts_remaining",
    "defteam_timeouts_remaining",
    "total_home_score",
    "total_away_score",
    "posteam_score",
    "defteam_score",
    "posteam_score_post",
    "defteam_score_post",
    "score_differential_post",
    "no_score_prob",
    "opp_fg_prob",
    "opp_safety_prob",
    "opp_td_prob",
    "fg_prob",
    "safety_prob",
    "extra_point_prob",
    "two_point_conversion_prob",
    "total_home_epa",
    "total_away_epa",
    "total_home_rush_epa",
    "total_away_rush_epa",
    "total_home_pass_epa",
    "total_away_pass_epa",
    "air_epa",
    "yac_epa",
    "comp_air_epa",
    "comp_yac_epa",
    "total_home_comp_air_epa",
    "total_away_comp_air_epa",
    "total_home_comp_yac_epa",
    "total_away_comp_yac_epa",
    "total_home_raw_air_epa",
    "total_away_raw_air_epa",
    "total_home_raw_yac_epa",
    "total_away_raw_yac_epa",
    "def_wp",
    "home_wp",
    "away_wp",
    "vegas_wpa",
    "vegas_home_wpa",
    "home_wp_post",
    "away_wp_post",
    "vegas_wp",
    "vegas_home_wp",
    "total_home_rush_wpa",
    "total_away_rush_wpa",
    "total_home_pass_wpa",
    "total_away_pass_wpa",
    "air_wpa",
    "yac_wpa",
    "comp_air_wpa",
    "comp_yac_wpa",
    "total_home_comp_air_wpa",
    "total_away_comp_air_wpa",
    "total_home_comp_yac_wpa",
    "total_away_comp_yac_wpa",
    "total_home_raw_air_wpa",
    "total_away_raw_air_wpa",
    "total_home_raw_yac_wpa",
    "total_away_raw_yac_wpa",
    "punt_blocked",
    "touchback",
]

In [ ]:
drop_columns2 = [
    "punt_attempt",
    "safety_player_name",
    "punt_inside_twenty",
    "kicker_player_name",
    "passing_yards",
    "interception_player_name",
    "lateral_kickoff_returner_player_id",
    "assist_tackle",
    "qb_hit_2_player_id",
    "penalty_team",
    "lateral_receiver_player_name",
    "lateral_recovery",
    "wind",
    "rush_touchdown",
    "defensive_two_point_conv",
    "receiver_jersey_number",
    "tackle_with_assist",
    "fumbled_1_player_name",
    "tackle_with_assist_2_player_id",
    "two_point_attempt",
    "drive_quarter_start",
    "lateral_receiving_yards",
    "end_yard_line",
    "defensive_extra_point_attempt",
    "pass_touchdown",
    "lateral_rusher_player_name",
    "rusher_id",
    "aborted_play",
    "drive_yards_penalized",
    "fumble_not_forced",
    "penalty_player_id",
    "tackle_for_loss_2_player_id",
    "drive_end_yard_line",
    "passer_id",
    "stadium_id",
    "sack_player_name",
    "punt_out_of_bounds",
    "tackle_with_assist_2_team",
    "surface",
    "kickoff_downed",
    "extra_point_attempt",
    "punt_fair_catch",
    "assist_tackle_2_player_name",
    "fumble_forced",
    "special_teams_play",
    "drive_ended_with_score",
    "half_sack_1_player_name",
    "stadium",
    "lateral_receiver_player_id",
    "lateral_sack_player_name",
    "play",
    "tackle_with_assist_1_player_id",
    "forced_fumble_player_2_team",
    "home_coach",
    "xyac_epa",
    "lateral_punt_returner_player_name",
    "fantasy",
    "solo_tackle_1_team",
    "drive_start_transition",
    "first_down",
    "game_stadium",
    "xyac_fd",
    "drive_play_count",
    "passer",
    "fumbled_1_player_id",
    "replay_or_challenge_result",
    "drive_real_start_time",
    "receiver_player_id",
    "solo_tackle_2_player_id",
    "fumbled_2_player_name",
    "qb_hit_1_player_name",
    "kickoff_attempt",
    "xyac_success",
    "season",
    "rush",
    "tackle_with_assist_2_player_name",
    "assist_tackle_2_team",
    "sack_player_id",
    "assist_tackle_1_team",
    "play_deleted",
    "rusher_jersey_number",
    "pass_oe",
    "return_team",
    "tackle_for_loss_2_player_name",
    "time_of_day",
    "end_clock_time",
    "tackle_with_assist_1_team",
    "home_score",
    "tackle_with_assist_1_player_name",
    "kickoff_inside_twenty",
    "own_kickoff_recovery_player_id",
    "lateral_reception",
    "qb_hit_1_player_id",
    "own_kickoff_recovery_td",
    "pass_defense_2_player_name",
    "jersey_number",
    "punter_player_name",
    "blocked_player_name",
    "pass_defense_1_player_name",
    "xyac_median_yardage",
    "st_play_type",
    "success",
    "penalty_player_name",
    "punt_returner_player_name",
    "return_touchdown",
    "blocked_player_id",
    "assist_tackle_1_player_id",
    "receiving_yards",
    "half_sack_2_player_name",
    "drive_game_clock_start",
    "rusher",
    "pass_defense_1_player_id",
    "touchdown",
    "assist_tackle_4_player_id",
    "lateral_return",
    "solo_tackle_2_team",
    "kickoff_in_endzone",
    "fumble_out_of_bounds",
    "return_yards",
    "punt_downed",
    "nfl_api_id",
    "defensive_extra_point_conv",
    "out_of_bounds",
    "lateral_interception_player_name",
    "lateral_rush",
    "interception_player_id",
    "assist_tackle_3_player_name",
    "pass_defense_2_player_id",
    "receiver_player_name",
    "away_score",
    "forced_fumble_player_2_player_name",
    "qb_hit_2_player_name",
    "order_sequence",
    "lateral_rusher_player_id",
    "punt_returner_player_id",
    "play_clock",
    "cpoe",
    "punt_in_endzone",
    "fantasy_player_name",
    "passer_player_name",
    "xyac_mean_yardage",
    "fixed_drive",
    "forced_fumble_player_1_player_name",
    "lateral_interception_player_id",
    "solo_tackle",
    "kickoff_out_of_bounds",
    "fumbled_2_player_id",
    "fumbled_1_team",
    "defensive_two_point_attempt",
    "spread_line",
    "drive_game_clock_end",
    "home_opening_kickoff",
    "fantasy_id",
    "forced_fumble_player_1_player_id",
    "away_coach",
    "fumbled_2_team",
    "kickoff_fair_catch",
    "half_sack_1_player_id",
    "receiver",
    "punter_player_id",
    "xpass",
    "replay_or_challenge",
    "rusher_player_name",
    "pass",
    "assist_tackle_1_player_name",
    "fixed_drive_result",
    "kickoff_returner_player_id",
    "forced_fumble_player_1_team",
    "half_sack_2_player_id",
    "temp",
    "id",
    "drive_end_transition",
    "passer_player_id",
    "tackle_for_loss_1_player_name",
    "field_goal_attempt",
    "lateral_punt_returner_player_id",
    "play_type_nfl",
    "drive_first_downs",
    "result",
    "receiver_id",
    "start_time",
    "name",
    "rusher_player_id",
    "passer_jersey_number",
    "fantasy_player_id",
    "tackle_for_loss_1_player_id",
    "own_kickoff_recovery_player_name",
    "drive_time_of_possession",
    "forced_fumble_player_2_player_id",
    "assist_tackle_2_player_id",
    "own_kickoff_recovery",
    "solo_tackle_1_player_name",
    "special",
    "lateral_sack_player_id",
    "lateral_rushing_yards",
    "assist_tackle_3_team",
    "drive_quarter_end",
    "location",
    "total",
    "rushing_yards",
    "total_line",
    "solo_tackle_1_player_id",
    "assist_tackle_4_player_name",
    "assist_tackle_4_team",
    "safety_player_id",
    "drive_start_yard_line",
    "kicker_player_id",
    "assist_tackle_3_player_id",
    "lateral_kickoff_returner_player_name",
    "kickoff_returner_player_name",
    "solo_tackle_2_player_name",
]

In [ ]:
drop_columns3 = [
    "drive_inside20",
    "penalty",
    "penalty_yards",
    "penalty_type",
    "drive_play_id_ended",
    "drive_play_id_started",
    "first_down_penalty",
    "fourth_down_converted",
    "fourth_down_failed",
    "fumble_lost",
    "fumble_recovery_1_player_id",
    "fumble_recovery_1_player_name",
    "fumble_recovery_1_team",
    "fumble_recovery_1_yards",
    "fumble_recovery_2_player_id",
    "fumble_recovery_2_player_name",
    "fumble_recovery_2_team",
    "fumble_recovery_2_yards",
    "pass_attempt",
    "qb_epa",
    "rush_attempt",
    "safety",
    "series_result",
    "series_success",
    "third_down_converted",
    "third_down_failed",
    "complete_pass",
    "incomplete_pass",
    "sack",
    "tackled_for_loss",
    "half_seconds_remaining",
    "game_half",
    # "weather",
    "air_yards",
    "yards_after_catch",
]

In [ ]:
drop_columns4 = ["cp", "weather"]

In [ ]:
# drop columns
combined_df.drop(drop_columns1, axis=1, inplace=True)
combined_df.drop(drop_columns2, axis=1, inplace=True)
combined_df.drop(drop_columns3, axis=1, inplace=True)
combined_df.drop(drop_columns4, axis=1, inplace=True)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
display(combined_df.head(10))
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

In [ ]:
combined_df.shape

## NA Clearance
Tok

In [ ]:
# detect NAs
for column in combined_df.columns:
    print(f"{column}: {sum(combined_df[column].isna())}")

In [ ]:
combined_df = combined_df.dropna()

In [ ]:
for column in combined_df.columns:
    print(f"{column}: {sum(combined_df[column].isna())}")

## Data Validity Check
Tok

In [ ]:
# plausibility check of the data sets
print("number of duplicates:", combined_df.duplicated().sum())
print("number of distinct teams ('posteam'):", combined_df['posteam'].nunique())
print("different 'defteam_type' attributes:", set(combined_df['posteam_type']))
print("different 'roof' attributes:", set(combined_df['roof']))
display(combined_df[(combined_df['yardline_100'] <= 0) | (combined_df['yardline_100'] >= 100)].shape)
display(combined_df[(combined_df['game_date'] < str(1999)) | (combined_df['game_date'] >= str(2024))].shape)
display(combined_df[(combined_df['game_seconds_remaining'] < 0) | (combined_df['game_seconds_remaining'] > 3600)].shape)
display(combined_df[(combined_df['down'] < 1) | (combined_df['down'] > 4)].shape)
display(combined_df[(combined_df['ydstogo'] < 1) | (combined_df['ydstogo'] > 99)].shape)
display(combined_df[combined_df['yards_gained'] > 99].shape)
display(combined_df[(combined_df['interception'] == 1) & (combined_df['play_type'] != 'pass')].shape)

In [ ]:
# pass plays with a QB scramble
display(combined_df[(combined_df['play_type'] == 'pass') & (combined_df['qb_scramble'] == 1)].shape)
display(combined_df[(combined_df['play_type'] == 'pass') & (combined_df['qb_scramble'] == 1)])

In [ ]:
# plays which were not first down but their first down rush or pass attribute is true
display(combined_df[(combined_df['first_down_rush'] == 1) & (combined_df['down'] != 1)].shape)
display(combined_df[(combined_df['first_down_rush'] == 1) & (combined_df['down'] != 1)])
display(combined_df[(combined_df['first_down_pass'] == 1) & (combined_df['down'] != 1)].shape)
display(combined_df[(combined_df['first_down_pass'] == 1) & (combined_df['down'] != 1)])

In [ ]:
# run plays where the QB was hit
display(combined_df[(combined_df['qb_hit'] == 1) & (combined_df['play_type'] != 'pass')].shape)
display(combined_df[(combined_df['qb_hit'] == 1) & (combined_df['play_type'] != 'pass')])

In [ ]:
print(combined_df.isnull().sum())


## Encoding of Categorical Features
Thilo

In [ ]:
# encode categorical features
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# create ColumnTransformer
encoder = ColumnTransformer(
    transformers=[
        # ('encoder', OneHotEncoder(drop='first'), ['roof']) ], 
        ('encoder', OneHotEncoder(drop='first'), ['posteam','posteam_type', 'roof', 'defteam']) ], 
        remainder='passthrough'  # include non-transformed columns
)
encoded_data= encoder.fit_transform(combined_df)
feature_names = [item.replace('encoder__', '').replace('remainder__', '') for item in encoder.get_feature_names_out()]

# convert preprocessed data to DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=feature_names)


In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
display(encoded_df.head())
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

In [ ]:
# remove remaining for modelling useless features
encoded_df.drop(["game_date", "desc"], axis=1, inplace=True)

## Correlation Analysis
Thilo

In [ ]:
import plotly.graph_objects as go
correlation_matrix = encoded_df.drop(['play_type'], axis=1).corr()
column_list=encoded_df.drop(['play_type'], axis=1).columns
fig = go.Figure(
    data=go.Heatmap(
        z=correlation_matrix,
        x=list(column_list),
        y=list(column_list),
        colorscale="rdylbu",
        zmin=-1,
        zmax=1

    )
)

fig.update_layout(
    showlegend=False, width=800, height=800, autosize=False, title="Correlation matrix"
)

fig.update_yaxes(showticklabels=False, autorange="reversed")
fig.update_xaxes(showticklabels=False)
fig.show()

In [ ]:
# removing highly correlated features (not both but one)
encoded_df.drop(['ep', 'wp', 'series', 'epa'], axis= 1, inplace= True)

In [ ]:
correlation_matrix = encoded_df.drop(['play_type'], axis=1).corr()
column_list=encoded_df.drop(['play_type'], axis=1).columns
fig = go.Figure(
    data=go.Heatmap(
        z=correlation_matrix,
        x=list(column_list),
        y=list(column_list),
        colorscale="rdylbu",
        zmin=-1,
        zmax=1
    )
)

fig.update_layout(
    showlegend=False, width=800, height=800, autosize=False, title="Correlation matrix"
)

fig.update_yaxes(showticklabels=False, autorange="reversed")
fig.update_xaxes(showticklabels=False)
fig.show()

## Outlier Removal
Joel

In [ ]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns

# set options to work with
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# show first 5 rows of dataframe
display(encoded_df.head())

# Convert the datatypes of the attributes in the DataFrame
for column in encoded_df.columns:
    try:
        encoded_df[column] = pd.to_numeric(encoded_df[column])
    except ValueError:
        encoded_df[column] = encoded_df[column].apply(str)

# Seperate boolean from numeric values, as they are irrelevant for the purpose filtering outliers
boolean_variables = [
    'posteam_ATL', 'posteam_BAL', 'posteam_BUF', 'posteam_CAR', 'posteam_CHI', 'posteam_CIN', 'posteam_CLE', 'posteam_DAL', 'posteam_DEN', 
    'posteam_DET', 'posteam_GB', 'posteam_HOU', 'posteam_IND', 'posteam_JAX', 'posteam_KC', 'posteam_LA', 'posteam_LAC', 'posteam_LV', 'posteam_MIA', 'posteam_MIN',
    'posteam_NE', 'posteam_NO', 'posteam_NYG', 'posteam_NYJ', 'posteam_PHI', 'posteam_PIT', 'posteam_SEA', 'posteam_SF', 'posteam_TB', 'posteam_TEN', 'posteam_WAS', 
    'roof_dome', 'roof_open', 'roof_outdoors', 'goal_to_go', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_scramble', 'first_down_rush', 
    'first_down_pass', 'interception', 'qb_hit', 'fumble', 'div_game',
    'posteam_type_home', 'defteam_ATL', 
    'defteam_BAL', 'defteam_BUF', 'defteam_CAR', 'defteam_CHI', 'defteam_CIN', 'defteam_CLE', 'defteam_DAL', 'defteam_DEN', 'defteam_DET', 'defteam_GB', 'defteam_HOU', 
    'defteam_IND', 'defteam_JAX', 'defteam_KC', 'defteam_LA', 'defteam_LAC', 'defteam_LV', 'defteam_MIA', 'defteam_MIN', 'defteam_NE', 'defteam_NO', 'defteam_NYG', 
    'defteam_NYJ', 'defteam_PHI', 'defteam_PIT', 'defteam_SEA', 'defteam_SF', 'defteam_TB', 'defteam_TEN', 'defteam_WAS'
    ]

# boolean_variables = encoded_df.select_dtypes(include='bool').columns.tolist()

for variable in boolean_variables:
    encoded_df[variable] = encoded_df[variable].astype(bool)

# 1. Create a Copy to work with
numeric_df = encoded_df.select_dtypes(include=['number']).copy()

print(numeric_df)

# 2. Checking ranges and distributions
ranges_df = pd.DataFrame(columns=['min','max','mean', 'median', 'quantile1', 'quantile3', 'iqr', 'lower', 'upper'])

for column in numeric_df.columns:
    min_values = numeric_df[column].min()
    max_values = numeric_df[column].max()
    mean = numeric_df[column].mean()
    median = numeric_df[column].median()

    #set quantile
    quantile_value = 0.25
    q1 = numeric_df[column].quantile(quantile_value)
    q3 = numeric_df[column].quantile(1-quantile_value)
    iqr = q3-q1
    lower_bound = q1 - 3.0 * iqr
    upper_bound = q3 + 3.0 * iqr
    

    ranges_df.loc[column] = [min_values, max_values, mean, median, q1, q3, iqr, lower_bound, upper_bound]

    # Display the distributions of each column + the quantiles
    # Histogram
    plt.hist(numeric_df[column], bins=20, alpha=0.7)
    plt.axvline(q1, color='red', linestyle='dashed', linewidth=2, label='quantile 1')
    plt.axvline(q3, color='blue', linestyle='dashed', linewidth=2, label='quantile 3')
    plt.axvline(lower_bound, color='black', linestyle='dashed', linewidth=2, label='lower_bound')
    plt.axvline(upper_bound, color='black', linestyle='dashed', linewidth=2, label='upper_bound')
    plt.legend()
    plt.title('Histogram: ' + column)
    plt.show()

# Summary of ranges
display(ranges_df)

# Analyzing the Diagrams the following statements can be made
# a) Most of the values outliers can be explained and are therefor meaningfull for the dataset
#       --> Get rid of attributes that are negligible for outlier identification
drop_columns = ['yardline_100', 'game_seconds_remaining', 'down']
numeric_df.drop(drop_columns, axis=1, inplace=True)

# b) the data is already well defined and does not contain too many outliers
#       --> Keep only the rows where the values are within 3.0 times the IQR from Q1 and Q3
ranges_updated_df = pd.DataFrame(columns=['min','max','mean', 'median'])
discarded_rows = pd.DataFrame(columns=encoded_df.columns)

# 3. Remove the Rows that hold outliers 
for column in numeric_df.columns:
    lower_bound = ranges_df['lower'][column]
    upper_bound = ranges_df['upper'][column]

    outliers = encoded_df.loc[(numeric_df[column] < lower_bound) | (numeric_df[column] > upper_bound)]
    discarded_rows = pd.concat([discarded_rows, outliers])
    encoded_df = encoded_df.loc[~((encoded_df[column] < lower_bound) | (encoded_df[column] > upper_bound))]
    numeric_df = numeric_df.loc[~((numeric_df[column]< lower_bound) | (numeric_df[column] > upper_bound))]

    # Histogram after removal of Extremal values
    plt.hist(numeric_df[column], bins=20, alpha=0.7)
    plt.title('Histogram: ' + column)
    plt.show()

    min_values = numeric_df[column].min()
    max_values = numeric_df[column].max()
    mean = numeric_df[column].mean()
    median = numeric_df[column].median()

    ranges_updated_df.loc[column] = [min_values, max_values, mean, median]

# 4. Display results
display(ranges_updated_df)
display(discarded_rows.head())
display(encoded_df.head())

# reset options
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

## Normalization
Thilo

In [ ]:
# undo transformation to boolean values for binary features
for variable in boolean_variables:
    encoded_df[variable] = encoded_df[variable].astype(int)

In [ ]:
numeric_features = [
    "yardline_100",
    "game_seconds_remaining",
    "down",
    "ydstogo",
    "score_differential",
    "td_prob",
    "wpa",
]

In [ ]:
# plot histograms to see the approximate distribution
for column in numeric_features:
   lower_bound = ranges_df['lower'][column]
   upper_bound = ranges_df['upper'][column]
   numeric_df = encoded_df.loc[~((encoded_df[column]< lower_bound) | (encoded_df[column] > upper_bound))]

   # Histogram after removal of Extremal values
   plt.hist(numeric_df[column], bins=20, alpha=0.7)
   plt.title('Histogram: ' + column)
   plt.show()

only the score_differential looks similar to a normal distribution. Therefore, it will be normalized using mean normalization while all other numerical features will be normalized using min-max-scaling

In [ ]:
numeric_features.remove('score_differential')

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
normalization= ColumnTransformer(
    transformers=[
        ('standardization', StandardScaler(),['score_differential']),
        ('minmax', MinMaxScaler(), numeric_features)
    ],
    remainder='passthrough'  # include non-transformed columns
    )

normalized_data= normalization.fit_transform(encoded_df)
feature_names = [item.replace('standardization__', '').replace('minmax__', '').replace('remainder__', '') for item in normalization.get_feature_names_out()]
normalized_df = pd.DataFrame(normalized_data, columns=feature_names)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
display(normalized_df.head())
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

## Splitting

In [ ]:
run_df= normalized_df[normalized_df['play_type']=="run"]
run_df.drop(["play_type"], axis=1, inplace=True)
pass_df= normalized_df[normalized_df['play_type']=="pass"]
pass_df.drop(["play_type"], axis=1, inplace=True)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
display(run_df.head())
display(pass_df.head())
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

run_df.to_csv("Data/run_df.csv")
pass_df.to_csv("Data/pass_df.csv")